mehr GTFS Daten beschaffen

man kann im code auch den check nach land == "US" rausnehmen und man bekommt Internationaler ergebnisse, warscheinlich overkill

In [ ]:
%pip install tqdm

In [2]:
import csv


# open file with all links
with open('sources.csv', newline='', encoding = "utf-8") as f:
    reader = csv.reader(f)
    data_file = list(reader)
    data_file.pop(0)


# filter the data so irrelevant rows are not downloaded
cleaned_list = []
for row in data_file:

    format = row[1]
    link = row[10]

    # for file naming later
    country = row[3]
    state = row[4]
    area = row[5]

    if len(state) == 0 or len(area) == 0: continue # if there are no correct names


    if len(link)==0: link = row[11] # if first link is not there take the second one thats provided
    if len(link)==0: continue # if it doenst exist either: skip
    if link[-4:] != ".zip": continue # only take files in the .zip format

    if format !=  "gtfs": continue # skip if wrong format
    if country != "US": continue # skip if wrong country

    res = [link, country, state, area]
    cleaned_list.append(res)
print(len(cleaned_list))
    


569


In [8]:
import requests as r
from tqdm import tqdm
import os

# könnte man multithreaden, muss aber nur einmal laufen
# requested einfach einmal jede URL und speichert das Ergebnis als ZIP
# Name ist aus Land-State-City zusammengesetzt

print("temporary directory for all raw downloaded data created (This can be deleted after everything is done)")
os.mkdir("temporary_zips")
print("starting download, will take around 2h")
for row in tqdm(cleaned_list):

    url = row[0]
    filename = row[1] + " " + row[2] +" " + row[3] + ".zip" # ex: US_Alabama_Birmingham
    filename = filename.replace(" ", "_") # replace whitespaces
    try:
        response = r.get(url, timeout=5) # get file
        
        open("temporary_zips/" + filename, "wb").write(response.content) # save zip
    except Exception as e:
        print(e)

temporary directory for all raw downloaded data created (This can be deleted after everything is done)


FileExistsError: [WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: 'temporary_zips'

In [21]:
import glob
import zipfile
from tqdm import tqdm
import os

# get all zip files in directory
all_files = glob.glob("temporary_zips/*")


if not os.path.isdir("data"): os.mkdir("data")

# unzip every file and save it
for file in tqdm(all_files):
    

# some files are not zip files
    try:    
        with zipfile.ZipFile(file, 'r') as zip_ref:

            # filename if the zip
            filename = os.path.basename(file)
            filename = filename[:-4]

            pathname = "data/" + filename

            print(pathname)
            os.mkdir(pathname) # create first dir level
            os.mkdir(pathname + "/gtfs") # create second one
            zip_ref.extractall(pathname + "/gtfs") # extract
            
    except Exception as e:
        print(e)

100%|██████████| 2/2 [00:00<00:00, 76.92it/s]

data/US_Maine_Casco_Bay
data/US_New_York_Canton
